In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',50)
#!pip install pgeocode
#Python library for high performance off-line querying of GPS coordinates
# import pgeocode
# nomi = pgeocode.Nominatim(country='IN', unique=True)

In [12]:
final_invoice_df = pd.read_csv('C:/Users/alams/Desktop/Titanic/Mahindra First Choice Dataset/Final_invoice.csv')
jtd_df = pd.read_csv('C:/Users/alams/Desktop/Titanic/Mahindra First Choice Dataset/JTD.csv')
customer_df = pd.read_excel('C:/Users/alams/Desktop/Titanic/Mahindra First Choice Dataset/Customer_Data.xlsx')

In [15]:
customer_df['Business Partner'].nunique()

555338

In [16]:
final_invoice_df.shape, jtd_df.shape, customer_df.shape

((492314, 59), (5619484, 10), (555338, 9))

# Total Column 78

In [17]:
customer_df.head()
customer_df.isnull().sum()/len(customer_df['Customer No.'])*100

Business Partner     0.000000
Customer No.         0.000000
Partner Type         0.014046
Data Origin          0.024490
Title                8.338525
Marital Status      97.112209
Occupation          97.378533
Date of Birth       97.579132
Death date          99.880613
dtype: float64

In [18]:
#Dropped Columns from Customer data, More than 90% are Null
customer_df = customer_df.drop(['Title','Marital Status','Occupation','Date of Birth','Death date'],axis=1)

In [19]:
customer_df['Partner Type'] = customer_df['Partner Type'].fillna(customer_df['Partner Type'].mode()[0])
customer_df['Data Origin'] = customer_df['Data Origin'].fillna(customer_df['Data Origin'].mode()[0])

In [20]:
customer_df.isnull().sum()

Business Partner    0
Customer No.        0
Partner Type        0
Data Origin         0
dtype: int64

In [21]:
final_invoice_df = final_invoice_df.drop(['Amt Rcvd From Custom','Amt Rcvd From Ins Co',
                                          'CGST(14%)','CGST(2.5%)', 'CGST(6%)','IGST(12%)','CGST(9%)',
                                          'IGST(18%)','IGST(28%)','IGST(5%)','SGST/UGST(14%)',
                                          'SGST/UGST(2.5%)','SGST/UGST(6%)','SGST/UGST(9%)',
                                          'TDS amount','Service Advisor Name','Outstanding Amt',
                                        'Total CGST','Total GST','Total IGST','Total SGST/UGST',
                                          'Policy no.','Cash /Cashless Type','Expiry Date','Gate Pass Date',
                                          'Insurance Company','Claim No.'
                                         ],axis=1)

In [22]:
final_invoice_df.shape

(492314, 32)

In [23]:
final_invoice_df["Invoice_DateTime"]=final_invoice_df["Invoice Date"]+' '+final_invoice_df["Invoice Time"]
final_invoice_df["JobCard_DateTime"]=final_invoice_df["JobCard Date"]+' '+final_invoice_df["JobCard Time"]
final_invoice_df['JobCard_DateTime'] = pd.to_datetime(final_invoice_df['JobCard_DateTime'],dayfirst=True)
final_invoice_df['Invoice_DateTime'] = pd.to_datetime(final_invoice_df['Invoice_DateTime'],dayfirst=True)
final_invoice_df["Service_Time"]=final_invoice_df["Invoice_DateTime"]-final_invoice_df["JobCard_DateTime"]

In [24]:
final_invoice_df["Service_Time"]

0        19 days 21:00:53
1         0 days 19:55:14
2         0 days 23:32:13
3         0 days 01:28:12
4         1 days 01:21:35
               ...       
492309    0 days 05:41:46
492310    0 days 06:24:37
492311    0 days 07:16:05
492312    0 days 03:05:11
492313    0 days 00:15:13
Name: Service_Time, Length: 492314, dtype: timedelta64[ns]

In [25]:
final_invoice_df.drop(['JobCard Date','JobCard Time','Invoice Date','Invoice Time'],axis = 1,inplace=True)

In [26]:
import pgeocode
nomi = pgeocode.Nominatim('IN')

In [27]:
nomi.query_postal_code('400059')

postal_code                                    400059
country code                                       IN
place_name        Marol Bazar, J.B. Nagar, Marol Naka
state_name                                Maharashtra
state_code                                         16
county_name                                    Mumbai
county_code                                       519
community_name                                 Mumbai
community_code                                    NaN
latitude                                      18.9292
longitude                                     72.8856
accuracy                                            1
Name: 0, dtype: object

In [28]:
final_invoice_df.columns

Index(['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.',
       'District', 'Gate Pass Time', 'Invoice No', 'Job Card No',
       'KMs Reading', 'Labour Total', 'Make', 'Misc Total', 'Model', 'ODN No.',
       'OSL Total', 'Order Type', 'Parts Total', 'Pin code', 'Plant',
       'Plant Name1', 'Print Status', 'Recovrbl Exp', 'Regn No',
       'Technician Name', 'Total Amt Wtd Tax.', 'Total Value', 'User ID',
       'Invoice_DateTime', 'JobCard_DateTime', 'Service_Time'],
      dtype='object')

In [29]:
final_invoice_df["Clean_CITY"]=nomi.query_postal_code(np.array(final_invoice_df["Pin code"].astype(str))).county_name
final_invoice_df["Clean_District"]=nomi.query_postal_code(np.array(final_invoice_df["Pin code"].astype(str))).state_name
final_invoice_df["Clean_Location_Area"]=nomi.query_postal_code(np.array(final_invoice_df["Pin code"].astype(str))).place_name

In [30]:
final_invoice_df[["Clean_CITY",'Clean_District',"Clean_Location_Area"]]

,Clean_CITY,Clean_District,Clean_Location_Area
0,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S."
1,Thane,Maharashtra,"Thane East, Kopri Colony"
2,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh"
3,Thane,Maharashtra,"Thane East, Kopri Colony"
4,Thane,Maharashtra,"Thane East, Kopri Colony"
...,...,...,...
492309,Tiruvallur,Tamil Nadu,"Ambattur Indl Estate, Athipattu"
492310,Kanchipuram,Tamil Nadu,"Adambakkam, Nilamangai Nagar"
492311,Kanchipuram,Tamil Nadu,"Lakshmipuram, Chromepet, Nehrunagar, Nagalkeni..."
492312,Chennai,Tamil Nadu,Velacheri


In [31]:
final_invoice_df.drop(["Area / Locality","CITY","District"],1,inplace=True)

In [32]:
final_invoice_df.shape, jtd_df.shape, customer_df.shape

((492314, 31), (5619484, 10), (555338, 4))

In [34]:
# # dropping unnecessary columns
# customer_data_df.drop(['Unnamed: 0'],axis=1,inplace=True)
final_invoice_df.drop(['Unnamed: 0','ODN No.','Technician Name','Regn No','Total Value'],axis=1,inplace=True)
jtd_df.drop(['Unnamed: 0','Labor Value Number'],axis=1,inplace=True)

In [35]:
final_invoice_df.columns

Index(['Cust Type', 'Customer No.', 'Gate Pass Time', 'Invoice No',
       'Job Card No', 'KMs Reading', 'Labour Total', 'Make', 'Misc Total',
       'Model', 'OSL Total', 'Order Type', 'Parts Total', 'Pin code', 'Plant',
       'Plant Name1', 'Print Status', 'Recovrbl Exp', 'Total Amt Wtd Tax.',
       'User ID', 'Invoice_DateTime', 'JobCard_DateTime', 'Service_Time',
       'Clean_CITY', 'Clean_District', 'Clean_Location_Area'],
      dtype='object')

In [ ]:
final_invoice_df.drop(['Unnamed: 0','ODN No.','Technician Name','Regn No','Total Value'],axis=1,inplace=True)

In [23]:
customer_df.columns

Index(['Business Partner', 'Customer No.', 'Partner Type', 'Data Origin'], dtype='object')

In [ ]:
jtd_df.drop(['Unnamed: 0','Labor Value Number'],axis=1,inplace=True)

In [84]:
jtd_df['Item Category'].value_counts()

P002    3250293
P001    1845451
P011     343288
P010     128823
G2TX      51629
Name: Item Category, dtype: int64

In [53]:
jtd_df.head()

,DBM Order,Order Item,Material,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,22385,180,OSLCC_SRV31,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87
1,23910,10,1DM279018170117,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13
2,23910,40,1IS150203BE01,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92
3,23910,60,1DM279001990173,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14
4,23910,70,1DM88552790S001SA,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63


In [46]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'P002'].value_counts()[:5]

OIL FILTER                    154387
URANIA SUPREMO CI 4 15W 40    122218
MFC PROLEUM CI4/SL 15W40       92091
AIR FILTER                     80211
WINDSHIELD LIQUID WASHER       55352
Name: Description, dtype: int64

In [47]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'P001'].value_counts()[:5]

Paid Service-Minor            201857
WASHING                       198483
Wheel Alignment               191209
Wheel Balancing (5 Wheels)     79969
MISC LABOR 0.5 HOURS           62982
Name: Description, dtype: int64

In [48]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'P011'].value_counts()[:5]

MISC_OTHER CHARGES                        270380
MISC_ON ROAD ASSISTANCE                    18991
MISC_COST OF WEIGHT (EACH)                 12529
MISC_PUC                                    9985
MISC_ON ROAD ASSISTANCE WITHOUT TOWING      6988
Name: Description, dtype: int64

In [49]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'P010'].value_counts()[:5]

PAINTING CHARGES            15239
REPAIRING CHARGES           12838
ENGINE FLUSH TREATMENT      11556
REMOVE REFITTING CHARGES     9944
Engine Flush                 9073
Name: Description, dtype: int64

In [53]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'G2TX'].unique()

array([nan], dtype=object)

In [65]:
jtd_df['Description'].loc[jtd_df['Item Category'] == 'G2TX']

722        NaN
979        NaN
980        NaN
1591       NaN
2339       NaN
          ... 
5619350    NaN
5619376    NaN
5619386    NaN
5619417    NaN
5619418    NaN
Name: Description, Length: 51629, dtype: object

In [26]:
final_invoice_df.head()

,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,Clean_CITY,Clean_District,Clean_Location_Area
0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S."
1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14,Thane,Maharashtra,"Thane East, Kopri Colony"
2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,233.14,BC01SA2,2015-01-03 11:12:57,2015-01-02 11:40:44,0 days 23:32:13,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh"
3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,0.00,BC01SA2,2015-01-03 11:40:44,2015-01-03 10:12:32,0 days 01:28:12,Thane,Maharashtra,"Thane East, Kopri Colony"
4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,2747.12,BC01SA2,2015-01-03 12:07:17,2015-01-02 10:45:42,1 days 01:21:35,Thane,Maharashtra,"Thane East, Kopri Colony"


In [27]:
customer_df.head()

,Business Partner,Customer No.,Partner Type,Data Origin
0,31,1,1.0,Z001
1,32,2,1.0,Z005
2,34,4,1.0,Z005
3,35,5,1.0,Z005
4,42,7,1.0,Z008


# Joining Invoice and Customer Data

In [29]:
invoice_customer = pd.merge(final_invoice_df,customer_df,how='left',on='Customer No.')

In [31]:
invoice_customer.item_catogory

,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,Clean_CITY,Clean_District,Clean_Location_Area,Business Partner,Partner Type,Data Origin
0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",NaN,NaN,NaN
1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN
2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,233.14,BC01SA2,2015-01-03 11:12:57,2015-01-02 11:40:44,0 days 23:32:13,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh",NaN,NaN,NaN
3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,0.00,BC01SA2,2015-01-03 11:40:44,2015-01-03 10:12:32,0 days 01:28:12,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN
4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,2747.12,BC01SA2,2015-01-03 12:07:17,2015-01-02 10:45:42,1 days 01:21:35,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN


In [52]:
jtd_grp.head()

,DBM Order,Item Category,Net value,Order Quantity
0,4,P001,3455.92,714.0
1,4,P002,11042.62,15.6
2,4,P010,2399.95,1.0
3,4,P011,100.00,1.0
4,5,P001,2310.47,283.0


# Joining Invoice_Customer With JTD

In [32]:
jtd_grp = jtd_df.groupby(["DBM Order","Item Category"],as_index=False).agg({"Net value":'sum',"Order Quantity":"sum"})

In [51]:
jtd_grp.columns

Index(['DBM Order', 'Item Category', 'Net value', 'Order Quantity'], dtype='object')

In [34]:
invoice_customer_jtd_grp = pd.merge(invoice_customer,jtd_grp,how="left",left_on="Job Card No",right_on="DBM Order")

In [35]:
invoice_customer_jtd_grp.shape

(936275, 33)

In [36]:
invoice_customer_jtd_grp.to_csv('Mahindra_cleaned_df.csv')

In [37]:
invoice_customer_jtd_grp.isnull().sum()

Cust Type                   0
Customer No.                0
Gate Pass Time              0
Invoice No                  0
Job Card No                 0
KMs Reading                 0
Labour Total                0
Make                        0
Misc Total                  0
Model                    3185
OSL Total                   0
Order Type                  0
Parts Total                 0
Pin code                    0
Plant                       0
Plant Name1                 0
Print Status                0
Recovrbl Exp                0
Total Amt Wtd Tax.          0
User ID                     0
Invoice_DateTime            0
JobCard_DateTime            0
Service_Time                0
Clean_CITY              94162
Clean_District          94162
Clean_Location_Area     94162
Business Partner       894749
Partner Type           894749
Data Origin            894749
DBM Order                  39
Item Category              39
Net value                  39
Order Quantity             39
dtype: int

In [38]:
invoice_customer_jtd_grp.dtypes

Cust Type                       object
Customer No.                    object
Gate Pass Time                  object
Invoice No                       int64
Job Card No                      int64
KMs Reading                      int64
Labour Total                   float64
Make                            object
Misc Total                     float64
Model                           object
OSL Total                      float64
Order Type                      object
Parts Total                    float64
Pin code                         int64
Plant                           object
Plant Name1                     object
Print Status                    object
Recovrbl Exp                   float64
Total Amt Wtd Tax.             float64
User ID                         object
Invoice_DateTime        datetime64[ns]
JobCard_DateTime        datetime64[ns]
Service_Time           timedelta64[ns]
Clean_CITY                      object
Clean_District                  object
Clean_Location_Area      

In [44]:
invoice_customer_jtd_grp.head()

,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,Clean_CITY,Clean_District,Clean_Location_Area,Business Partner,Partner Type,Data Origin,DBM Order,Item Category,Net value,Order Quantity
0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",NaN,NaN,NaN,168303.0,P001,1203.14,73.0
1,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",NaN,NaN,NaN,168303.0,P002,2348.75,6.5
2,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",NaN,NaN,NaN,168303.0,P010,500.06,1.0
3,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN,173997.0,P001,804.26,170.0
4,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14,Thane,Maharashtra,"Thane East, Kopri Colony",NaN,NaN,NaN,173997.0,P011,197.03,1.0


In [ ]:
invoice_customer_jtd_grp['Description'] = jtd_grp['Description']

In [99]:
#to check numeric columns
invoice_customer_jtd_grp._get_numeric_data()
#to check categorical data 
g = invoice_customer_jtd_grp.select_dtypes(include =["object","bool_"])
g.columns

Index(['Cust Type', 'Customer No.', 'Gate Pass Time', 'Make', 'Model',
       'Order Type', 'Plant', 'Plant Name1', 'Print Status', 'User ID',
       'Clean_CITY', 'Clean_District', 'Clean_Location_Area',
       'Business Partner', 'Data Origin', 'Item Category'],
      dtype='object')

In [8]:
final_invoice_df.isna().mean().round(4) * 100

Unnamed: 0               0.00
Amt Rcvd From Custom    40.07
Amt Rcvd From Ins Co    40.07
Area / Locality          4.85
CGST(14%)               40.07
                        ...  
Total GST               40.07
Total IGST              40.07
Total SGST/UGST         40.07
Total Value             40.07
User ID                  0.00
Length: 59, dtype: float64